In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
standing_url =  'https://fbref.com/en/comps/9/Premier-League-Stats'

In [2]:
data = requests.get(standing_url)
soup = BeautifulSoup(data.text)

In [3]:
standing_table = soup.select('table.stats_table')[0]
links = standing_table.find_all('a')
links = [l.get('href') for l in links]

In [4]:
links = [l for l in links if '/squads/' in l]

In [5]:
team_urls = [f'https://fbref.com{l}' for l in links]

In [6]:
team_url = team_urls[0]
data = requests.get(team_url)
matches = pd.read_html(data.text, match = 'Scores & Fixtures')
matches[0].head()

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squad

In [ ]:
soup = BeautifulSoup(data.text)
link = soup.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f'https://fbref.com{links[0]}')
shooting = pd.read_html(data.text, match = 'Shooting')[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches[0].merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK' ,'PK', 'PKatt']], on='Date')

In [ ]:
years = list(range(2022, 2020, -1))

In [ ]:
all_matches = []
standing_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
import time
for year in years:
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standing_table = soup.select('table.stats_table')[0]
    links = standing_table.find_all('a')
    links = [l.get('href') for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f'https://fbref.com{l}' for l in links]
    previous_season = soup.select('a.prev')[0].get('href')
    standing_url = f'https://fbref.com/{previous_season}'

    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace('-Stats','').replace('-', ' ')

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = 'Scores & Fixtures')[0]

        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get('href') for l in links]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match = 'Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on = 'Date')
        except ValueError:
            continue
        
        team_data = team_data[team_data['Comp'] == 'Premier League']
        team_data['Season'] = year
        team_data['Team'] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [ ]:
team_url.split('/')[-1].replace('-Stats', '').replace('-', ' ')

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv('matches.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c51dd7fb-2a41-426e-92c7-0ccbf159f7cc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>